# Multi-Task Language Processing with LangChain and Featherless
This notebook demonstrates how to create a flexible multi-task pipeline using LangChain and Featherless AI. We'll show how to:
- Use different models for different tasks
- Chain multiple operations together
- Process text through translation and sentiment analysis
## Prerequisites
1. Sign up for an account at [Featherless](https://featherless.ai/register)
2. Subscribe to a plan and get your API key from [API Keys](https://featherless.ai/account/api-keys)
## Setup
First, let's import the required libraries and set up our API key.

In [ ]:
!pip install langchain langchain-openai

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import os
# Set your API key
FEATHERLESS_API_KEY= "your-api-key-here" # Replace with actual API key
# Alternatively, you can set it as an environment variable
# FEATHERLESS_API_KEY = os.getenv("FEATHERLESS_API_KEY")

## Model Configuration
We'll use different models for different tasks to optimize performance. 

In [ ]:
# Define models for different tasks
classification_llm = ChatOpenAI(
    api_key="{FEATHERLESS_API_KEY}",
    base_url="https://api.featherless.ai/v1",
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
)

translation_llm = ChatOpenAI(
    api_key="{FEATHERLESS_API_KEY}",
    base_url="https://api.featherless.ai/v1",
    model="mistralai/Mistral-Nemo-Instruct-2407",
)

## Prompt Templates
Define templates for our translation and classification tasks

In [ ]:
# Define prompt templates
translation_prompt = PromptTemplate.from_template(
    "Translate the following sentence from English to French:\n{input}"
)
classification_prompt = PromptTemplate.from_template(
    "Classify the sentiment of the following text as positive, negative, or neutral:\n{input}"
)

## Task Definition
Create individual tasks for translation and classification that can be chained together.

In [ ]:
# Translation task
translation_task = RunnableLambda(
    lambda input: {
        "task": "Translation",
        "translated_text": translation_llm.invoke(translation_prompt.format(input=input["text"])).content  # Extract content
    }
)

# Classification task
classification_task = RunnableLambda(
    lambda input: {
        "task": "Classification",
        "translated_text": input["translated_text"],  # Passing the translated text
        "classification_result": classification_llm.invoke(classification_prompt.format(input=input["translated_text"])).content
    }
)

## Workflow Creation and Execution
Chain the tasks together and run the workflow with a sample input.

In [ ]:
# Chain the tasks together
workflow = RunnablePassthrough() | translation_task | classification_task

# Run the workflow
result = workflow.invoke({"text": "I love using Featherless."})
print(result)